In [479]:
%reload_ext autoreload
%autoreload 2
%xmode Verbose

Exception reporting mode: Verbose


Checking out the basic functions manually

In [480]:
import utils as helper
import pandas as pd

In [481]:
sample_data_raw = [[22,125,13],
               [275,15,3],
               [23,15,4],
               [123,17,3]]
sample_data = pd.DataFrame(sample_data_raw)
print ("oth col is_numeric? :", helper.is_numeric(sample_data, 0))
print (helper.label_counts(sample_data))
print ("###########")
print (helper.most_probable_label(sample_data))
print (helper.most_probable_label(sample_data.iloc[:1, :]))

oth col is_numeric? : True
3     2
13    1
4     1
Name: 2, dtype: int64
###########
3
13


General Child Parent relationships below

In [482]:
class Dog():
    def __init__(self):
        pass
    
    def bark(self):
        print ("barking")

In [483]:
class Pup(Dog):
    def __init__(self):
        pass
    def cry(self):
        print ("pups don't cry, steriotyper")

In [484]:
P = Pup()
P.bark()

barking


Checking the criterion class

In [485]:
import criterion

In [486]:
g = criterion.Gini()
curr = g.impurity(sample_data)
print (curr)
print (g.info_gain(sample_data[0:2],
                   sample_data[2:],
                   curr))
# We would get the info gain to be less than .44
# as we did bad job in splitting

0.625
0.125


Testing Questions.

In [487]:
from question import Question

In [488]:
col = 1
wedge = 17
q = Question(col, wedge)
print("mein col=", col, "mein valu=", wedge, "se >= hai!")
q.divide_on_question(sample_data)[0]

mein col= 1 mein valu= 17 se >= hai!


,0,1,2
0,22,125,13
3,123,17,3


In [489]:
print("mein col=", col, "mein valu=", wedge, "se < hai!")
q.divide_on_question(sample_data)[1]

mein col= 1 mein valu= 17 se < hai!


,0,1,2
1,275,15,3
2,23,15,4


Testing some DT Functions

In [490]:
from dt import DecisionTreeClassifier, DecisionTreeRegression

In [491]:
d = DecisionTreeClassifier()

In [492]:
d.train(sample_data)

pritin the most probablie clss 3
pritin the most probablie clss 4
pritin the most probablie clss 13


In [493]:
out = d._best_split(sample_data)
print (out)
left, right = out[1].divide_on_question(sample_data)

(0.375, <question.Question object at 0x7f0884407438>)


In [494]:
d.root.l_child.get_prediction()

3

In [495]:
sample_data

,0,1,2
0,22,125,13
1,275,15,3
2,23,15,4
3,123,17,3


In [496]:
d.root.question.col_index, d.root.question.wedge

(0, 123)

In [497]:
print ("left: True Rows")
left

left: True Rows


,0,1,2
1,275,15,3
3,123,17,3


In [498]:
print ("right: False Rows")
right

right: False Rows


,0,1,2
0,22,125,13
2,23,15,4


Test the nodes that are used to build the DT

In [499]:
X_test_raw = [[222,12],
              [24,15]]
X_test = pd.DataFrame(X_test_raw)

In [500]:
nodes.DecisionNode(out[1], None, None)

In [501]:
sample_data

,0,1,2
0,22,125,13
1,275,15,3
2,23,15,4
3,123,17,3


In [502]:
d.predict(X_test)

,0
0,3
1,4


In [503]:
dr = DecisionTreeRegression()

In [504]:
dr.train(sample_data)

In [505]:
dr.predict(X_test)

,0
0,3.0
1,8.5


In [506]:
X_test

,0,1
0,222,12
1,24,15


In [507]:
i = 4. + 13
i / 2

8.5

In [508]:
import criterion
from question import Question
from nodes import DecisionNode, PredNodeClassify, PredNodeRegress

class DecisionTree:
    """you don't want to instantiate this
    object directly, use a derived class.
    """
    def __init__(self, 
                 method, 
                 max_depth, 
                 max_points):
        self.method = method
        self.max_depth = max_depth
        self.max_points = max_points
        self.criterion = self._make_metric_class()
        self.DecisionNode = DecisionNode
        self.root = None
        
    def train(self, rows):
        pass
        self.root = self._build_tree(rows, 0)

    def _make_metric_class(self):
        """This function helps in modifying the criterion
        object on the fly, given classification or regression
        """
        if self.method=='gini':
            return criterion.Gini()

        elif self.method=='entropy':
            return criterion.Entropy()
        
        elif self.method=='std':
            return criterion.STD()
    
    def _best_split(self, rows):
        '''Finds the best Question (using info_gain) that 
        splits the rows into left and right buckets.

        Returns a Question (records the col_index and the value.)
        @ bad performance when we have really large number of rows.
        Can improve by first sorting and then doing a binary search
        '''
        previous_impurity = self.criterion.impurity(rows)
        BestInfoGain = 0 # find the best col and val to split the rows
        best_question = None
        num_rows, num_cols = rows.shape
        for col in range(num_cols - 1): # last col is the label 
                                       # rows have same len
            for row in range(num_rows):
                val = rows.iloc[row, col]         # this val may be the wedge
                                                  # value for the question
                q = Question(col, val)
                left, right = q.divide_on_question(rows)
                if left.shape[0] == 0 or right.shape[0] == 0:
                    continue           # ignore the case when no splitting
                InfoGain = self.criterion.info_gain(left, right, previous_impurity)
                # if best info gain crossed save state or save Question
                if InfoGain >= BestInfoGain:
                    BestInfoGain = InfoGain
                    best_question = q
                    if previous_impurity == InfoGain:
                        return BestInfoGain, best_question # already best
        return BestInfoGain, best_question
    
    def _build_tree(self, rows, depth):
        '''This function would be recursively called
        We first set the base case where we check if the
        the depth of tree is less than some number or
        we don't have any difference in labels
        @add support for regression
        '''
        if depth == self.max_depth: # forced leaf
            return self.PredictionNode(rows)

        gain, question = self._best_split(rows)
        if gain == 0:               # we have encountered a leaf
            return self.PredictionNode(rows)

        left, right = question.divide_on_question(rows)
        left_branch = self._build_tree(left, depth + 1)
        right_branch = self._build_tree(right, depth + 1)
        return self.DecisionNode(question, left_branch, right_branch)
    
    def predict(self, rows):
        # for each row in df, apply self._predict_one
        return rows.apply(self._predict_one, axis=1).to_frame()

    def _predict_one(self, row, node=None):
        if isinstance(node, type(None)):
            node = self.root
        if isinstance(node, self.PredictionNode): # tree end
            return node.get_prediction()
        # Else we have a DecisionNode
        if node.question.is_satified(row):
            return self._predict_one(row, node.l_child)
        else:
            return self._predict_one(row, node.r_child)
    

class DecisionTreeClassifier(DecisionTree):
    """Instantiate this class for a Decision
    Tree for classification tasks.
    """
    def __init__(self,
                 method="gini",
                 max_depth=2,
                 max_points=None):
        self.PredictionNode = PredNodeClassify    # Prediction node is 
                                            # different in case of Regression
        super().__init__(method,
                         max_depth,
                         max_points)

class DecisionTreeRegression(DecisionTree):
    """Instantiate this class for a Decision
    Tree for Regression tasks.
    """
    def __init__(self,
                 method="std",
                 max_depth=2,
                 max_points=None):
        self.PredictionNode = PredNodeRegress    # Prediction node is 
                                                 # different in case of Regression
        super().__init__(method,
                         max_depth,
                         max_points)